<a href="https://colab.research.google.com/github/dhenypatungka/Utilities/blob/main/VideoSwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown ## 🌄 1. Install Dependencies
!git clone https://github.com/dhenypatungka/MultiSwap.git
%cd MultiSwap
!wget https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx ./models
!pip install yt-dlp
!pip install -r requirements.txt

In [ ]:
# @markdown ## 🌄 Image Source
# @markdown Upload your face photo, max 2048 px
import os
from google.colab import files
import shutil


# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  os.rename(filename, '/content/image.jpg')

In [ ]:
# @markdown ## 🌄 Download YouTube Video As Target
!pip install yt-dlp
import os

#@markdown #### Find YouTube video
#@markdown ``https://youtu.be/twX1GZBoBmY`` ⛔ <br>
#@markdown ``https://www.youtube.com/watch?v=twX1GZBoBmY`` ✅

!rm -df youtube.mp4

#@markdown ___
from urllib import parse as urlparse
YOUTUBE_URL = 'https://www.youtube.com/watch?v=S_ttsuSHEuk' #@param {type:"string"}
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

# remove previous input video
!rm -f /content/video.mp4

#@markdown ___
#@markdown #### Trim (seconds)
start = 3 #@param {type:"integer"}
end = 128 #@param {type:"integer"}
interval = end - start

# Download the YouTube video using yt-dlp
!yt-dlp --format "b[ext=mp4]" --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Cut the video using FFmpeg
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 /content/video.mp4

# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

In [ ]:
# @markdown ## 🌄 Swap Video

!python /content/MultiSwap/run.py \
--frame-processor face_swapper face_enhancer \
--target "/content/video.mp4" \
--source "/content/image.jpg" \
--output "/content/swapped.mp4" \
--keep-fps \
--execution-provider cuda \
--temp-frame-format png \
--temp-frame-quality 100 \
--reference-face-position 0

######################################################################################################################################
# run.py [options]

#  -h, --help                                                                 show this help message and exit
#  -s SOURCE_PATH, --source SOURCE_PATH                                       select an source image
#  -t TARGET_PATH, --target TARGET_PATH                                       select an target image or video
#  -o OUTPUT_PATH, --output OUTPUT_PATH                                       select output file or directory
#  --frame-processor FRAME_PROCESSOR [FRAME_PROCESSOR ...]                    frame processors (choices: face_swapper, face_enhancer, ...)
#  --keep-fps                                                                 keep target fps
#  --keep-frames                                                              keep temporary frames
#  --skip-audio                                                               skip target audio
#  --many-faces                                                               process every face
#  --reference-face-position REFERENCE_FACE_POSITION                          position of the reference face
#  --reference-frame-number REFERENCE_FRAME_NUMBER                            number of the reference frame
#  --similar-face-distance SIMILAR_FACE_DISTANCE                              face distance used for recognition
#  --temp-frame-format {jpg,png}                                              image format used for frame extraction
#  --temp-frame-quality [0-100]                                               image quality used for frame extraction
#  --output-video-encoder {libx264,libx265,libvpx-vp9,h264_nvenc,hevc_nvenc}  encoder used for the output video
#  --output-video-quality [0-100]                                             quality used for the output video
#  --max-memory MAX_MEMORY                                                    maximum amount of RAM in GB
#  --execution-provider {cpu} [{cpu} ...]                                     available execution provider (choices: cpu, ...)
#  --execution-threads EXECUTION_THREADS                                      number of execution threads
#  -v, --version                                                              show program's version number and exit

In [ ]:
# @markdown ## 🌄 Download Video
from google.colab import files
finalVideo = "/content/swapped.mp4"
files.download(finalVideo)
print("Download Result")